In [ ]:
!pip install pandas


Developed a python script to scrape most voted questions from Stack Overflow using the requests library.
Coded the script to convert questions into CSV file and avoided repetition of questions using the pandas library.
Programmed it further to create a question bank for other languages.

In [ ]:
!pip install requests-html

     |████████████████████████████████| 92kB 7.6MB/s 
     |████████████████████████████████| 81kB 7.6MB/s 
     |████████████████████████████████| 81kB 7.1MB/s 
  Created wheel for parse: filename=parse-1.19.0-cp37-none-any.whl size=24592 sha256=d81dd0a98af5d2f7f81865f47b58b72418164b17b4dc2177eb9ba2e791c71dd1
  Stored in directory: /root/.cache/pip/wheels/c0/39/ea/e2fd678bd130953f5438470b8dfa529f00787e9b8b92b27467
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp37-none-any.whl size=13501 sha256=c14e1ace17cf52c9eca1a320d4ff0330ea4de08e3791b591ce95ce2e737ee6d3
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
Successfully built parse fake-useragent
ERROR: pyppeteer 0.2.5 has requirement urllib3<2.0.0,>=1.25.8, but you'll have urllib3 1.24.3 which is incompatible.
  Found existing installation: importlib-metadata 4.5.0
    Uninstalling importlib-metadata-4.5.0:
      Successfully uninstalled importlib-metadata-

In [ ]:
import requests
from requests_html import HTML
import time
import pandas as pd

In [ ]:
base_url='https://stackoverflow.com/questions/tagged/'
tag="python"
querry_filter="Votes"
url=f"{base_url}{tag}?tab={querry_filter}"
url


'https://stackoverflow.com/questions/tagged/python?tab=Votes'

In [ ]:
r=requests.get(url)
html_str=r.text

In [ ]:
html=HTML(html=html_str)

question_summaries=html.find(".question-summary")
print(question_summaries)

[<Element 'div' class=('question-summary',) id='question-summary-231767'>, <Element 'div' class=('question-summary',) id='question-summary-419163'>, <Element 'div' class=('question-summary',) id='question-summary-394809'>, <Element 'div' class=('question-summary',) id='question-summary-100003'>, <Element 'div' class=('question-summary',) id='question-summary-82831'>, <Element 'div' class=('question-summary',) id='question-summary-38987'>, <Element 'div' class=('question-summary',) id='question-summary-89228'>, <Element 'div' class=('question-summary',) id='question-summary-273192'>, <Element 'div' class=('question-summary',) id='question-summary-522563'>, <Element 'div' class=('question-summary',) id='question-summary-952914'>, <Element 'div' class=('question-summary',) id='question-summary-136097'>, <Element 'div' class=('question-summary',) id='question-summary-509211'>, <Element 'div' class=('question-summary',) id='question-summary-176918'>, <Element 'div' class=('question-summary'

In [ ]:
question_summaries[0].text

"11219\nvotes\n40answers\n2.5m views\nWhat does the “yield” keyword do?\nWhat is the use of the yield keyword in Python? What does it do? For example, I'm trying to understand this code1: def _get_child_candidates(self, distance, min_dist, max_dist): if self._leftchild ...\npython iterator generator yield coroutine\nasked Oct 23 '08 at 22:21\nAlex. S.\n127k1616 gold badges5050 silver badges6161 bronze badges"

In [ ]:
columns=['votes','vote_title','num_ans','views','question','short_description','tags','date','user','user_details']

In [ ]:
# curr_row=list(question_summeries[0].text.split("\n"))
# print(curr_row)

# row_data=dict(zip(columns,curr_row))
# row_data

In [ ]:
key_names = ['question', 'votes', 'tags']
classes_needed = ['.question-hyperlink', '.vote', '.tags']
this_question_element = question_summaries[0]
this_question_element.find('.question-hyperlink', first=True).text
this_question_element.find('.vote', first=True).text.replace('\nvotes', '')
datas = []

In [ ]:
def clean_scraped_data(text, keyname=None):
    if keyname == 'votes':
        return text.replace('\nvotes', '')
    return text

In [ ]:
for q_el in question_summaries:
    question_data = {}
    for i, _class in enumerate(classes_needed):
        sub_el = q_el.find(_class, first=True)
        keyname = key_names[i]
        question_data[keyname] = clean_scraped_data(sub_el.text, keyname=keyname)
    datas.append(question_data)

In [ ]:
datas[0]

{'question': 'What does the “yield” keyword do?',
 'tags': 'python iterator generator yield coroutine',
 'votes': '11219'}

In [ ]:
def parse_tagged_page(html):
    question_summaries = html.find(".question-summary")
    key_names = ['question', 'votes', 'tags']
    classes_needed = ['.question-hyperlink', '.vote', '.tags']
    datas = []
    for q_el in question_summaries:
        question_data = {}
        for i, _class in enumerate(classes_needed):
            sub_el = q_el.find(_class, first=True)
            keyname = key_names[i]
            question_data[keyname] = clean_scraped_data(sub_el.text, keyname=keyname)
        datas.append(question_data)
    return datas

In [ ]:
def extract_data_from_url(url):
    r = requests.get(url)
    if r.status_code not in range(200, 299):
        return []
    html_str = r.text
    html = HTML(html=html_str)
    datas = parse_tagged_page(html)
    return datas

In [ ]:
def scrape_tag(tag = "python", query_filter = "Votes", max_pages=50, pagesize=25):
    base_url = 'https://stackoverflow.com/questions/tagged/'
    datas = []
    for p in range(max_pages):
        page_num = p + 1
        url = f"{base_url}{tag}?tab={query_filter}&page={page_num}&pagesize={pagesize}"
        datas += extract_data_from_url(url)
        time.sleep(1.2)
    return datas

In [ ]:
datas = scrape_tag(tag='python')

In [ ]:
len(datas)

2500

In [ ]:
df = pd.DataFrame(datas)
df.head()

,question,votes,tags
0,What does the “yield” keyword do?,11219,python iterator generator yield coroutine
1,What does if __name__ == “__main__”: do?,6849,python namespaces main python-module idioms
2,Does Python have a ternary conditional operator?,6709,python operators ternary-operator conditional-...
3,What are metaclasses in Python?,6204,python oop metaclass python-class python-datam...
4,How do I check whether a file exists without e...,6176,python file file-exists


In [ ]:

df.shape


(2500, 3)

In [ ]:
df.to_csv("python_stackoverflow.csv", index=False)
